AMOS classes:

    spleen: 1
    right kidney: 2
    left kidney: 3
    gallbladder: 4
    esophagus: 5
    liver: 6
    stomach: 7
    aorta: 8
    inferior vena cava: 9
    pancreas: 10
    right adrenal gland: 11
    left adrenal gland: 12
    duodenum: 13
    bladder: 14
    prostate/uterus: 15


In [1]:
import pandas as pd
import os
import numpy as np
import nibabel as nib
from tqdm import tqdm
import glob

from monai.transforms import (
    Compose,
    LoadImaged,
)

from monai.data import (
    Dataset,
    load_decathlon_datalist,
)

 missing cuda symbols while dynamic loading
 cuFile initialization failed


In [2]:
transforms = Compose(
            [
                LoadImaged(keys=["image", "label"], image_only=False),
                ]
)

In [17]:
data_dir = "/nvmescratch/ceib/multi_organ_seg/AMOS/amos22/"
split_json = "dataset.json"

datasets = data_dir + split_json
datalist = load_decathlon_datalist(datasets, True, "training")[:200]
val_files = load_decathlon_datalist(datasets, True, "validation")[:100]
train_ds = Dataset(
    data=datalist,
    transform=transforms,
)
val_ds = Dataset(data=val_files, transform=transforms)

amos = train_ds + val_ds

In [20]:
rare_masks = {}
c = 0 
for scan in tqdm(amos):
    if len(np.unique(scan['label'])) != 16:
        rare_masks[c] = scan['label_meta_dict']['filename_or_obj'][-16:]
    c += 1

100%|██████████| 300/300 [17:12<00:00,  3.44s/it]


In [22]:
len(rare_masks)

39

In [21]:
rare_masks

{5: 'amos_0009.nii.gz',
 6: 'amos_0010.nii.gz',
 13: 'amos_0021.nii.gz',
 25: 'amos_0044.nii.gz',
 31: 'amos_0052.nii.gz',
 33: 'amos_0057.nii.gz',
 39: 'amos_0067.nii.gz',
 47: 'amos_0079.nii.gz',
 61: 'amos_0104.nii.gz',
 67: 'amos_0115.nii.gz',
 76: 'amos_0129.nii.gz',
 79: 'amos_0134.nii.gz',
 84: 'amos_0142.nii.gz',
 85: 'amos_0143.nii.gz',
 89: 'amos_0153.nii.gz',
 91: 'amos_0156.nii.gz',
 92: 'amos_0158.nii.gz',
 100: 'amos_0172.nii.gz',
 106: 'amos_0181.nii.gz',
 111: 'amos_0190.nii.gz',
 124: 'amos_0225.nii.gz',
 132: 'amos_0245.nii.gz',
 144: 'amos_0279.nii.gz',
 148: 'amos_0294.nii.gz',
 176: 'amos_0376.nii.gz',
 193: 'amos_0402.nii.gz',
 204: 'amos_0029.nii.gz',
 210: 'amos_0056.nii.gz',
 213: 'amos_0070.nii.gz',
 220: 'amos_0112.nii.gz',
 221: 'amos_0117.nii.gz',
 222: 'amos_0120.nii.gz',
 229: 'amos_0150.nii.gz',
 236: 'amos_0191.nii.gz',
 253: 'amos_0247.nii.gz',
 272: 'amos_0311.nii.gz',
 273: 'amos_0313.nii.gz',
 276: 'amos_0323.nii.gz',
 287: 'amos_0356.nii.gz'}

In [38]:
lack_irrelevant_masks = {}
c = 0
for case in tqdm(rare_masks.keys()):
    unique_values = np.unique(amos[case]['label'])
    if (len(unique_values)==15 and 14 not in unique_values) or (len(unique_values)==15 and 15 not in unique_values) or (len(unique_values)==14 and 14 not in unique_values and 15 not in unique_values):
        lack_irrelevant_masks[c] = amos[case]['label_meta_dict']['filename_or_obj'][-16:]
    c += 1

100%|██████████| 39/39 [02:31<00:00,  3.88s/it]


In [39]:
len(lack_irrelevant_masks)

11

In [40]:
lack_irrelevant_masks

{0: 'amos_0009.nii.gz',
 3: 'amos_0044.nii.gz',
 8: 'amos_0104.nii.gz',
 11: 'amos_0134.nii.gz',
 13: 'amos_0143.nii.gz',
 14: 'amos_0153.nii.gz',
 16: 'amos_0158.nii.gz',
 20: 'amos_0225.nii.gz',
 26: 'amos_0029.nii.gz',
 30: 'amos_0117.nii.gz',
 34: 'amos_0247.nii.gz'}